In [1]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_sur = pd.read_csv(r'C:\Users\pc\Downloads\BASE_SUR.txt', sep=r'¡', engine='python')
base_sur.head()

## Eliminación de variables: 
base_sur2 = base_sur[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_sur3 = base_sur2.dropna()

In [3]:
## Group by por nit para calificacion
b4 = base_sur3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_sur4 = pd.merge(base_sur3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_sur4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_sur4['y'] = pd.cut(x=base_sur4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [4]:
dataTypeSeries = base_sur4.dtypes
dataTypeSeries

cat = base_sur4.select_dtypes(include=['object','category'])
cuanti = base_sur4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_sur5=cuanti.join(cat)
print(base_sur5.dtypes)

base_sur5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')
base_sur5 = pd.read_csv(r'C:\Users\pc\Desktop\base_sur_process.csv', sep=',', engine='python')


X = base_sur5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_sur5['y']


CUCONUSE                            int64
SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [5]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Results****
Accuracy: 83.8545%
Log Loss: 2.525707071140132
DecisionTreeClassifier
****Results****
Accuracy: 81.6664%
Log Loss: 0.46712190083409433
RandomForestClassifier
****Results****
Accuracy: 77.3920%
Log Loss: 0.6632683613724217
MLPClassifier
****Results****
Accuracy: 81.5788%
Log Loss: 0.4701264371463453
LogisticRegression
****Results****
Accuracy: 77.1594%
Log Loss: 0.5801760843045976
GaussianNB
****Results****
Accuracy: 75.5858%
Log Loss: 1.2926128511488975


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [10]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.797684  0.819759  0.808570  23524.0  24175.0  0.903839
1             0.774944  0.784966  0.779923  29693.0  30077.0  0.858081
2             0.534000  0.500365  0.516636  16432.0  15397.0  0.756058
avg / total   0.725780  0.729573  0.727482  69649.0  69649.0  0.853484,              precision    recall  f1-score  support     pred       AUC
0             0.780006  0.839823  0.808810  23524.0  25328.0  0.904900
1             0.783527  0.759329  0.771238  29692.0  28775.0  0.851963
2             0.533162  0.504382  0.518373  16432.0  15545.0  0.756532
avg / total   0.723270  0.726367  0.724270  69648.0  69648.0  0.851131,              precision    recall  f1-score  support     pred       AUC
0             0.800847  0.835742  0.817923  23524.0  24549.0  0.910232
1             0.753932  0.771757  0.762740  29692.0  30394.0  0.838084
2             0.515063  0.460930  0.486495  16432.0  14705.0  0.726849
avg